# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
from scipy.stats import norm
import sys
import pandas as pd
import pickle

### Helper Fuctions

In [2]:
%store -r analysis_hf
sys.path.insert(0, analysis_hf)
import observationDrop as od

# Set Directory Paths

In [3]:
# Directory path to load formatted Pathology Data
%store -r path_dataDir

# Directory path to save Calcualted Analysis Data
%store -r path_CalData

# [1] Loading

### Loading TAU Pathology Data

In [4]:
# path_TAU
with open(os.path.join(path_dataDir, 'path_TAU.pkl'), 'rb') as f:
    path_TAU = pickle.load(f)
f.close()

### Loading TDP Pathology Data

In [5]:
# path_TDP
with open(os.path.join(path_dataDir, 'path_TDP.pkl'), 'rb') as f:
    path_TDP = pickle.load(f)
f.close()

### Loading pathCoM

In [6]:
# pathCoM
with open(os.path.join(path_dataDir, 'pathCoM.pkl'), 'rb') as f:
    pathCoM = pickle.load(f)
f.close()

### Loading Covariance Matrices

In [7]:
# covTAU
with open(os.path.join(path_CalData, 'covTAU.pkl'), 'rb') as f:
    covTAU = pickle.load(f)
f.close()

# covTDP
with open(os.path.join(path_CalData, 'covTDP.pkl'), 'rb') as f:
    covTDP = pickle.load(f)
f.close()

# covTAU_gt_TDP
with open(os.path.join(path_CalData, 'covTAU_gt_TDP.pkl'), 'rb') as f:
    covTAU_gt_TDP = pickle.load(f)
f.close()

# covTDP_gt_TAU
with open(os.path.join(path_CalData, 'covTDP_gt_TAU.pkl'), 'rb') as f:
    covTDP_gt_TAU = pickle.load(f)
f.close()

# covTAU_gt_TDP_raw
with open(os.path.join(path_CalData, 'covTAU_gt_TDP_raw.pkl'), 'rb') as f:
    covTAU_gt_TDP_raw = pickle.load(f)
f.close()

# covTDP_gt_TAU_raw
with open(os.path.join(path_CalData, 'covTDP_gt_TAU_raw.pkl'), 'rb') as f:
    covTDP_gt_TAU_raw = pickle.load(f)
f.close()

### Loading LabelNames

In [8]:
with open(os.path.join(path_CalData, 'LabelNames.pkl'), 'rb') as f:
    LabelNames = pickle.load(f)
f.close()

# [2] Drop Regions with Few Observations

### This is done after we calculate covariance matrix, to account for the case where TAU_missing_index and TDP_missing_index does not match. Makes TAU_gt_TDP, TDP_gt_TAU hard to compute if we exclude these regions in path_TAU, path_TDP before we compute covariance matrix

## Define Number of Observation Threshold

In [9]:
obs_thresh = 3

## Modify pathCoM 
### (20, 3, 2) --> (40, 3) / first 20 rows {L} and last 20 rows {R} / Order same as pathNamesRaw repeated 2 times

In [10]:
pathCoM = np.vstack((pathCoM[:, :, 0], pathCoM[:, :, 1]))

## (1) Get Index of Label where Pathology Data is Missing or smaller than prefixed number of Observation --> Therefore is excluded in the 3D mapping

In [11]:
# TAU
TAU_missing_index = od.pathObsThresh(path_TAU, obs_thresh)

# TDP
TDP_missing_index = od.pathObsThresh(path_TDP, obs_thresh)

## (2) Drop these Regions in Covariance Matrices

In [12]:
# Modify the covMatlist data so exclude these regions
# Rows
covTAU_Drop = np.delete(covTAU, TAU_missing_index, axis = 0) # TAU Cov Mat
covTDP_Drop = np.delete(covTDP, TDP_missing_index, axis = 0) # TDP Cov Mat
covTAU_gt_TDP_Drop = np.delete(covTAU_gt_TDP, TAU_missing_index, axis = 0) # TAU_gt_TDP Cov Mat
covTDP_gt_TAU_Drop = np.delete(covTDP_gt_TAU, TDP_missing_index, axis = 0) # TDP_gt_TAU Cov Mat
covTAU_gt_TDP_raw_Drop = np.delete(covTAU_gt_TDP_raw, TAU_missing_index, axis = 0) # covTAU_gt_TDP_raw
covTDP_gt_TAU_raw_Drop = np.delete(covTDP_gt_TAU_raw, TDP_missing_index, axis = 0) # covTDP_gt_TAU_raw

# Columns
covTAU_Drop = np.delete(covTAU_Drop, TAU_missing_index, axis = 1) # TAU Cov Mat
covTDP_Drop = np.delete(covTDP_Drop, TDP_missing_index, axis = 1) # TDP Cov Mat
covTAU_gt_TDP_Drop = np.delete(covTAU_gt_TDP_Drop, TAU_missing_index, axis = 1) # TAU_gt_TDP Cov Mat
covTDP_gt_TAU_Drop = np.delete(covTDP_gt_TAU_Drop, TDP_missing_index, axis = 1) # TDP_gt_TAU Cov Mat
covTAU_gt_TDP_raw_Drop = np.delete(covTAU_gt_TDP_raw_Drop, TAU_missing_index, axis = 1) # covTAU_gt_TDP_raw
covTDP_gt_TAU_raw_Drop = np.delete(covTDP_gt_TAU_raw_Drop, TDP_missing_index, axis = 1) # covTDP_gt_TAU_raw

## (3) Drop these Regions in Pathology Data (TAU, TDP)

In [13]:
path_TAU_Drop = np.delete(path_TAU, TAU_missing_index, axis = 1)
path_TDP_Drop = np.delete(path_TDP, TDP_missing_index, axis = 1)

## (4) Drop these Regions in LabelNames

In [14]:
# Generate the pathNamesRaw list for each TAU and TDP
pathNames_TAU_Drop = np.delete(LabelNames, TAU_missing_index)
pathNames_TDP_Drop = np.delete(LabelNames, TDP_missing_index)

## (5) Drop these Regions in pathCoM

In [15]:
# Generate CoM for each TAU and TDP
CoM_TAU_Drop = np.delete(pathCoM, TAU_missing_index, axis = 0)
CoM_TDP_Drop = np.delete(pathCoM, TDP_missing_index, axis = 0)

# [3] Saving

### Save Index of Label where Pathology Data is Missing or smaller than prefixed number of Observation (TAU, TDP)

In [16]:
# TAU_missing_index
with open(os.path.join(path_CalData, 'TAU_missing_index.pkl'), 'wb') as f:
    pickle.dump(TAU_missing_index, f)
f.close()

# TDP_missing_index
with open(os.path.join(path_CalData, 'TDP_missing_index.pkl'), 'wb') as f:
    pickle.dump(TDP_missing_index, f)
f.close()

### Save Dropped Covariance Matrices

In [17]:
# covTAU_Drop
with open(os.path.join(path_CalData, 'covTAU_Drop.pkl'), 'wb') as f:
    pickle.dump(covTAU_Drop, f)
f.close()

# covTDP_Drop
with open(os.path.join(path_CalData, 'covTDP_Drop.pkl'), 'wb') as f:
    pickle.dump(covTDP_Drop, f)
f.close()

# covTAU_gt_TDP_Drop
with open(os.path.join(path_CalData, 'covTAU_gt_TDP_Drop.pkl'), 'wb') as f:
    pickle.dump(covTAU_gt_TDP_Drop, f)
f.close()

# covTDP_gt_TAU_Drop
with open(os.path.join(path_CalData, 'covTDP_gt_TAU_Drop.pkl'), 'wb') as f:
    pickle.dump(covTDP_gt_TAU_Drop, f)
f.close()

# covTAU_gt_TDP_raw_Drop
with open(os.path.join(path_CalData, 'covTAU_gt_TDP_raw_Drop.pkl'), 'wb') as f:
    pickle.dump(covTAU_gt_TDP_raw_Drop, f)
f.close()

# covTDP_gt_TAU_raw_Drop
with open(os.path.join(path_CalData, 'covTDP_gt_TAU_raw_Drop.pkl'), 'wb') as f:
    pickle.dump(covTDP_gt_TAU_raw_Drop, f)
f.close()

### Save Dropped Pathology Data (TAU, TDP)

In [18]:
# path_TAU_Drop
with open(os.path.join(path_dataDir, 'path_TAU_Drop.pkl'), 'wb') as f:
    pickle.dump(path_TAU_Drop, f)
f.close()

# path_TDP_Drop
with open(os.path.join(path_dataDir, 'path_TDP_Drop.pkl'), 'wb') as f:
    pickle.dump(path_TDP_Drop, f)
f.close()

### Save Dropped LabelNames (TAU, TDP)

In [19]:
# pathNames_TAU_Drop
with open(os.path.join(path_CalData, 'pathNames_TAU_Drop.pkl'), 'wb') as f:
    pickle.dump(pathNames_TAU_Drop, f)
f.close()

# pathNames_TDP_Drop
with open(os.path.join(path_CalData, 'pathNames_TDP_Drop.pkl'), 'wb') as f:
    pickle.dump(pathNames_TDP_Drop, f)
f.close()

### Save Dropped pathCoM (TAU, TDP)

In [20]:
# CoM_TAU_Drop
with open(os.path.join(path_dataDir, 'CoM_TAU_Drop.pkl'), 'wb') as f:
    pickle.dump(CoM_TAU_Drop, f)
f.close()

# CoM_TDP_Drop
with open(os.path.join(path_dataDir, 'CoM_TDP_Drop.pkl'), 'wb') as f:
    pickle.dump(CoM_TDP_Drop, f)
f.close()